In [31]:
import pandas as pd
import numpy as np

# preparing dataset :

In [33]:
df = pd.read_csv("loan_data.csv")
df.head()

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1


In [35]:
df=df.drop(columns=["person_education","loan_intent"])

In [37]:
df['person_gender']=np.where(df['person_gender']=="female",0,1)
df['person_home_ownership']=np.where(df['person_home_ownership']=="RENT",0,1)
df['previous_loan_defaults_on_file']=np.where(df['previous_loan_defaults_on_file']=="No",0,1)

In [39]:
df.head()

,person_age,person_gender,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,0,71948.0,0,0,35000.0,16.02,0.49,3.0,561,0,1
1,21.0,0,12282.0,0,1,1000.0,11.14,0.08,2.0,504,1,0
2,25.0,0,12438.0,3,1,5500.0,12.87,0.44,3.0,635,0,1
3,23.0,0,79753.0,0,0,35000.0,15.23,0.44,2.0,675,0,1
4,24.0,1,66135.0,1,0,35000.0,14.27,0.53,4.0,586,0,1


In [41]:
df.shape

(45000, 12)

In [43]:
y=df['loan_status']
x=df.drop('loan_status',axis=1)

In [45]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.82,random_state=58)

**1. Logistic Regression model:**

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# scale features
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# train model
lr = LogisticRegression(max_iter=1000)
model = lr.fit(x_train_scaled, y_train)

# evaluate
score = model.score(x_test_scaled, y_test)
print(f"{score * 100:.2f} %")

88.88 %


**2. XGBoost model:**

In [59]:
from xgboost import XGBClassifier

model = XGBClassifier(
    n_estimators=300,
    learning_rate=0.1,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

model.fit(x_train, y_train)

score = model.score(x_test, y_test)
print(f"{score * 100:.2f} %")

92.20 %


# Splitting dataset between banks :

In [61]:
# Shuffle dataset to avoid bias
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split into chunks
bank1 = df.iloc[0:10000]
bank2 = df.iloc[10000:20000]
bank3 = df.iloc[20000:30000]
real_time_test = df.iloc[30000:45000]

# Save to CSV
bank1.to_csv("bank1_dataset.csv", index=False)
bank2.to_csv("bank2_dataset.csv", index=False)
bank3.to_csv("bank3_dataset.csv", index=False)
real_time_test.to_csv("real_time_testing_dataset.csv", index=False)